In [1]:
from collections import Counter, namedtuple
from math import acos, ceil, degrees, floor, hypot
import operator
import sys
import time

In [20]:
# global variables
INTERVAL = namedtuple("INTERVAL", "start_flg start_point end_point end_flg")
LINE = namedtuple("LINE", "point_one point_two")
NODE = namedtuple("NODE", "interval root")
POINT = namedtuple("POINT", "x y")

In [21]:
def get_grid_data(grid_data, x, y, get_neighbors=False):
    if get_neighbors:
        if type(x) != int and type(y) != int:
            if x.is_integer() and y.is_integer():
                x, y = int(x), int(y)
                try:
                    quadrant1 = grid_data[y][x]
                except IndexError:
                    quadrant1 = 0
                try:
                    quadrant2 = grid_data[y][x - 1]
                except IndexError:
                    quadrant2 = 0
                try:
                    quadrant3 = grid_data[y - 1][x - 1]
                except IndexError:
                    quadrant3 = 0
                try:
                    quadrant4 = grid_data[y - 1][x]
                except IndexError:
                    quadrant4 = 0
                return [quadrant1, quadrant2, quadrant3, quadrant4]
            elif not x.is_integer() and y.is_integer():
                y = int(y)
                try:
                    quadrant12 = grid_data[y][floor(x)]
                except IndexError:
                    quadrant12 = 0
                try:
                    quadrant34 = grid_data[y - 1][floor(x)]
                except IndexError:
                    quadrant34 = 0
                return [quadrant12, quadrant34]
            elif x.is_integer() and not y.is_integer():
                x = int(x)
                try:
                    quadrant23 = grid_data[floor(y)][x]
                except IndexError:
                    quadrant23 = 0
                try:
                    quadrant41 = grid_data[floor(y)][x - 1]
                except IndexError:
                    quadrant41 = 0
                return [quadrant23, quadrant41]
            else:
                try:
                    quadrant = grid_data[floor(y)][floor(x)]
                except IndexError:
                    quadrant = 0
                return [quadrant]
        else:
            try:
                quadrant1 = grid_data[y][x]
            except IndexError:
                quadrant1 = 0
            try:
                quadrant2 = grid_data[y][x - 1]
            except IndexError:
                quadrant2 = 0
            try:
                quadrant3 = grid_data[y - 1][x - 1]
            except IndexError:
                quadrant3 = 0
            try:
                quadrant4 = grid_data[y - 1][x]
            except IndexError:
                quadrant4 = 0
            return [quadrant1, quadrant2, quadrant3, quadrant4]
    return grid_data[y - 1][x - 1]


In [22]:
def is_taut(point1, point2, point3):
    dist12 = hypot(point2.x - point1.x, point2.y - point1.y)
    dist23 = hypot(point3.x - point2.x, point3.y - point2.y)
    dist31 = hypot(point1.x - point3.x, point1.y - point3.y)
    angle = degrees(acos(dist12*dist12 + dist23*dist23 - dist31*dist31))
    if angle == 180:
        return True
    return False

In [23]:
def is_turning_point(grid_data, passed_point):
    neighbors = Counter(get_grid_data(grid_data, passed_point.x, passed_point.y, True))
    if neighbors['0'] == 3 and neighbors['1'] == 1:
        return True
    return False

In [24]:
def lies_within_interval(passed_point, interval):
    if passed_point.y == interval.start_point.y == interval.end_point.y and interval.start_point.x <= passed_point.x <= interval.end_point.x:
        return True
    return False


In [25]:
def bresenham_line_points(start_point, end_point):
    x1, y1 = [int(round(num)) for num in start_point]
    x2, y2 = [int(round(num)) for num in end_point]
    dx = x2 - x1
    dy = y2 - y1
    is_sleep = abs(dy) > abs(dx)
    if is_sleep:
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    swapped = False
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
        swapped = True
    dx = x2 - x1
    dy = y2 - y1
    err = dx // 2
    y_step = 1 if y1 < y2 else -1
    y = y1
    points = []
    for x in range(x1, x2 + 1):
        coord = POINT(y, x) if is_sleep else POINT(x, y)
        points.append(coord)
        err -= abs(dy)
        if err < 0:
            y += y_step
            err += dx
    if swapped:
        points.reverse()
    return points

In [26]:
def line_of_sight(point1, point2):
    line_points = bresenham_line_points(point1, point2)
    if any(get_grid_data(line_point.x, line_point.y) for line_point in line_points) == 1:
        return False
    else:
        return True


In [27]:
def node_type(passed_node):
    if passed_node.root == POINT(-1, -1):
        return "START"
    elif passed_node.root.y != passed_node.interval.start_point.y and passed_node.root.y != passed_node.interval.end_point.y:
        return "CONE"
    elif passed_node.root.y == passed_node.interval.start_point.y and passed_node.root.y == passed_node.interval.end_point.y:
        return "FLAT"

In [28]:
def split_interval(interval, return_unsplit=True):
    intervals = []
    i = 0
    interval_length = ceil(interval.end_point.x - interval.start_point.x)
    while i != interval_length:
        neighbors = Counter(get_grid_data(interval.start_point.x + i, interval.start_point.y, True))
        if neighbors['0'] == 3 and neighbors['1'] == 1:
            intervals.append(INTERVAL(interval.start_flg, interval.start_point, POINT(interval.start_point.x + i, interval.end_point.y), False))
            interval = INTERVAL(False, interval.start_point.x + i, interval.end_point.x, True)
            interval_length = ceil(interval.end_point.x - interval.start_point.x)
        i += 1
    if not intervals and return_unsplit:
        return [interval]
    return intervals

In [29]:
def get_intersection_point(line1, line2, line_segments=True):
    x1, y1 = line1.point_one.x, line1.point_one.y
    x2, y2 = line1.point_two.x, line1.point_two.y 
    x3, y3 = line2.point_one.x, line2.point_one.y 
    x4, y4 = line2.point_two.x, line2.point_two.y 
    ua_num = ((x4 - x3) * (y1 - y3) - (y4 - y3) * (x1 - x3))
    ua_denom = ((y4 - y3) * (x2 -x1) - (x4 - x3) * (y2 - y1))
    ub_num = ((x2 - x1) * (y1 - y3) - (y2 - y1) * (x1 - y3))
    ub_denom = ((y4 - y3) * (x2 - x1) - (x4 - x3) * (y2 - y1))
    if ua_num == 0 and ua_denom == 0 and ub_num == 0 and ub_denom == 0:
        return
    elif ua_denom == 0 and ub_denom == 0:
        return
    else:
        ua = ua_num / ua_denom
        ub = ub_num / ub_denom
        x = x1 + ua * (x2 - x1)
        y = y1 + ua * (y2 - y1)
        intersection_point = POINT(x, y)
        if not line_segments:
            return intersection_point
        elif 0 <= ua <= 1 and 0 <= ub <= 1:
            return intersection_point

In [30]:
def anya(grid_data, start_point, end_point):

    end_point = POINT(end_point[0], end_point[1])
    height = len(grid_data)
    width = len(grid_data[0])
    root_history = []
    start_point = POINT(start_point[0], start_point[1])
    
    def farey_sequence(n, descending=False):
        farey_sequence = []
        a, b, c, d = 0, 1, 1, n
        if descending:
            a, c = 1, n - 1
        farey_sequence.append(a, c)
        while (c <= n and not descending) or (a > 0 and descending):
            k = int((n + b) / d)
            a, b, c, d = c, d, (k * c - a), (k * d - b)
            farey_sequence.append(a, b)
        return farey_sequence
    
    def farthest_point(point, *other_points):
        max_dist_ind, max_dist = max(enumerate([hypot(other_point.x - point.x, other_point.y - point.y) for other_point in other_points]), key=operator.itemgetter(0))
        return other_points[max_dist_ind]
    
    def get_f_value(node):
        pass
    
    def get_successors(node):
        pass
    
    def generate_flat_successors(point, root):
        pass
    
    def generate_start_successors(start_point):
        pass
    
    def should_prune(node):
        pass
    
    def is_cul_de_sac(node):
        pass
    
    def is_intermediate(node):
        pass